# Sequelize
[sequelize](https://sequelize.org/) - один из ORM-фреймворков для Node.js. Будем использовать его с расширением [sequelize-typescript](https://www.npmjs.com/package/sequelize-typescript).

Ещё можете посмотреть использование с [Nest.js](https://docs.nestjs.com/recipes/sql-sequelize).

Чтобы запустить этот notebook - [tslab](https://github.com/yunabe/tslab)

## Инициализация

Установка:
```
$ yarn add sequelize@6 typescript sequelize-typescript pg pg-hstore reflect-metadata
$ yarn add --dev @types/node @types/validator

a
```
`pg` и `pg-hstore` для подключения к postgresql.

In [1]:
import { Sequelize } from 'sequelize-typescript';

Первое действие - создание объекта `Sequelize`. Он создается один раз на всё приложение.

См. опции: https://sequelize.org/docs/v6/getting-started/

In [2]:
let sequelize = new Sequelize({
    host: 'localhost',
    dialect: 'postgres',
    username: 'postgres',
    password: 'localdbpass',
    port: 5432,
    database: 'orm-demo'
})

Проверка подключения

In [3]:
await sequelize.authenticate();

Executing (default): SELECT 1+1 AS result


^ по умолчанию sequelize логирует получающиеся SQL-запросы в консоль ^.

Это нужно отключить, если у вас выполняется очень много запросов, т.к. логирование добавляет некоторый overhead.

## Модели

1 модель sequelize = 1 класс = 1 таблица в БД.

## Определение моделей

In [4]:
import { 
    Table,
    Column,
    Model,
    PrimaryKey,
    AllowNull,
    CreatedAt,
    UpdatedAt,
    AutoIncrement,
    DataType,
    Default,
} from 'sequelize-typescript';
import { UUIDV4 } from 'sequelize'

In [5]:
@Table
class User extends Model {
    @PrimaryKey
    @Default(UUIDV4)
    @Column(DataType.UUID)
    declare uuid: string;

    @AllowNull(false)
    @Column(DataType.STRING(256))
    declare fio: string;

    @Column(DataType.TEXT)
    declare bio: string;
}

* Атрибут класса, помеченный декоратором `@Column` = столбец таблицы
  * Иногда typescript может вывести тип столбца из типа атрибута, но не в `tslab`
  * `@Default` - значение по умолчанию
  * `@AllowNull` - по умолчанию `true`, надо явно ставить `false`, чтобы сгенерировать `NOT NULL`
  * `declare` стал нужен с какой-то недавней версии typescript
* По умолчанию во все модели добавляется `id SERIAL PRIMARY KEY`, если нигде не повешен декоратор `@PrimaryKey`
  * Поэтому в последних версиях TypeScript есть трудности с переопределением атрибута `id`. См. https://github.com/microsoft/TypeScript/issues/51515
* Также автоматически добавляются и устанавливаются атрибуты:
  * `createdAt` - дата создания строчки
  * `updatedAt` - дата последнего удаления строчки
  * Это можно выключить так: `@Table({ timestamps: false })` (но наверное не нужно)
* Имя таблицы по умолчанию - `"Users"` (!)

### Синхронизация моделей с БД
Модели, созданные в качестве классов TypeScript, никуда не попадают. Надо отдельным действием перенести их в БД.

Для этого нужно добавить их к объекту sequelize:

In [6]:
sequelize.addModels([User])
// Или - https://www.npmjs.com/package/sequelize-typescript#configuration

Метод `sequelize.sync({ force: true })` пересоздает таблицы в БД (т.е. дропает всё и создает заново)

In [7]:
await sequelize.sync({ force: true })
null

Executing (default): DROP TABLE IF EXISTS "Users" CASCADE;
Executing (default): SELECT DISTINCT tc.constraint_name as constraint_name, tc.constraint_schema as constraint_schema, tc.constraint_catalog as constraint_catalog, tc.table_name as table_name,tc.table_schema as table_schema,tc.table_catalog as table_catalog,tc.initially_deferred as initially_deferred,tc.is_deferrable as is_deferrable,kcu.column_name as column_name,ccu.table_schema  AS referenced_table_schema,ccu.table_catalog  AS referenced_table_catalog,ccu.table_name  AS referenced_table_name,ccu.column_name AS referenced_column_name FROM information_schema.table_constraints AS tc JOIN information_schema.key_column_usage AS kcu ON tc.constraint_name = kcu.constraint_name JOIN information_schema.constraint_column_usage AS ccu ON ccu.constraint_name = tc.constraint_name WHERE constraint_type = 'FOREIGN KEY' AND tc.table_name = 'Users' AND tc.table_catalog = 'orm-demo'
Executing (default): DROP TABLE IF EXISTS "Users" CASCADE;
E

Без `force` будут созданы только несозданные таблицы:

In [8]:
await sequelize.sync()
null

Executing (default): SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'Users'
Executing (default): SELECT i.relname AS name, ix.indisprimary AS primary, ix.indisunique AS unique, ix.indkey AS indkey, array_agg(a.attnum) as column_indexes, array_agg(a.attname) AS column_names, pg_get_indexdef(ix.indexrelid) AS definition FROM pg_class t, pg_class i, pg_index ix, pg_attribute a WHERE t.oid = ix.indrelid AND i.oid = ix.indexrelid AND a.attrelid = t.oid AND t.relkind = 'r' and t.relname = 'Users' GROUP BY i.relname, ix.indexrelid, ix.indisprimary, ix.indisunique, ix.indkey ORDER BY i.relname;


### Создание моделей
Создадим инстанс модели:

In [9]:
let user = User.build({ fio: 'Корытов Павел Валерьевич' })
user

User {
  dataValues: {
    uuid: 'c17bc685-8775-4a88-9035-3e1d8f3ee4a9',
    fio: 'Корытов Павел Валерьевич'
  },
  _previousDataValues: { fio: undefined },
  uniqno: 1,
  _changed: Set(1) { 'fio' },
  _options: { isNewRecord: true, _schema: null, _schemaDelimiter: '' },
  isNewRecord: true
}


Мы создали объект, но он пока находится только в коде! Чтобы сохранить объект в БД, можно вызвать метод `save`:

In [10]:
await user.save()

Executing (default): INSERT INTO "Users" ("uuid","fio","createdAt","updatedAt") VALUES ($1,$2,$3,$4) RETURNING "uuid","fio","bio","createdAt","updatedAt";
User {
  dataValues: {
    uuid: 'c17bc685-8775-4a88-9035-3e1d8f3ee4a9',
    fio: 'Корытов Павел Валерьевич',
    updatedAt: 2023-11-08T23:35:31.193Z,
    createdAt: 2023-11-08T23:35:31.193Z,
    bio: null
  },
  _previousDataValues: {
    fio: 'Корытов Павел Валерьевич',
    uuid: 'c17bc685-8775-4a88-9035-3e1d8f3ee4a9',
    bio: null,
    createdAt: 2023-11-08T23:35:31.193Z,
    updatedAt: 2023-11-08T23:35:31.193Z
  },
  uniqno: 1,
  _changed: Set(0) {},
  _options: { isNewRecord: true, _schema: null, _schemaDelimiter: '' },
  isNewRecord: false
}


При вызове метода `save` поставились значения `createdAt` и `updatedAt`

In [11]:
user.createdAt

2023-11-08T23:35:31.193Z


Аналогично обновление:

In [12]:
user.bio = 'Программист ОИС'

Программист ОИС


(ничего не произошло)

In [13]:
await user.save()
console.log(user.createdAt,'|', user.updatedAt)

Executing (default): UPDATE "Users" SET "bio"=$1,"updatedAt"=$2 WHERE "uuid" = $3
2023-11-08T23:35:31.193Z | 2023-11-08T23:35:31.268Z


Есть shorthand-ы:

In [14]:
let user1 = await User.create({ fio: 'Азаревич Артём Дмитриевич' }); // build + save
null

Executing (default): INSERT INTO "Users" ("uuid","fio","createdAt","updatedAt") VALUES ($1,$2,$3,$4) RETURNING "uuid","fio","bio","createdAt","updatedAt";


In [15]:
await user1.update({ bio: 'Аспирант каф. МОЭВМ' }) // обновление полей + save
null

Executing (default): UPDATE "Users" SET "bio"=$1,"updatedAt"=$2 WHERE "uuid" = $3


Как создать много человек:

In [16]:
let GROUP_1303 = [
    'Депрейс Александр',
    'Коренев Данил',
    'Кузнецов Николай',
    'Смирнов Дмитрий',
    'Новак Полина'
]
let group1303 = await User.bulkCreate(
    GROUP_1303.map((fio) => ({ fio, bio: 'Студент группы 1303' }))
)
group1303.length

Executing (default): INSERT INTO "Users" ("uuid","fio","bio","createdAt","updatedAt") VALUES ('626e53d9-84d9-4964-b1bd-fb95ff807ede','Депрейс Александр','Студент группы 1303','2023-11-08 23:35:31.718 +00:00','2023-11-08 23:35:31.718 +00:00'),('1717ca0e-4d7e-4921-8e99-2ec72dcc87c5','Коренев Данил','Студент группы 1303','2023-11-08 23:35:31.718 +00:00','2023-11-08 23:35:31.718 +00:00'),('f82eeb99-fc89-4b42-b302-c3e0f2116c22','Кузнецов Николай','Студент группы 1303','2023-11-08 23:35:31.718 +00:00','2023-11-08 23:35:31.718 +00:00'),('30e492cd-68b5-48b7-b1d3-0247a1b1c22f','Смирнов Дмитрий','Студент группы 1303','2023-11-08 23:35:31.718 +00:00','2023-11-08 23:35:31.718 +00:00'),('b75eb344-cefb-4f25-a757-00c411be738b','Новак Полина','Студент группы 1303','2023-11-08 23:35:31.718 +00:00','2023-11-08 23:35:31.718 +00:00') RETURNING "uuid","fio","bio","createdAt","updatedAt";
5


Если нужно создать *очень* много человек, можно сэкономить время и память, не создавая инстансы `User` на каждую добавленную строчку:

In [17]:
let GROUP_1381 = [
    'Исайкин Георгий',
    'Тарасов Константин',
    'Васильева Ольга',
    'Возмитель Влас'
]
await User.bulkCreate(
    GROUP_1381.map((fio) => ({ fio, bio: 'Студент группы 1381' })),
    { returning: false }
)
null

Executing (default): INSERT INTO "Users" ("uuid","fio","bio","createdAt","updatedAt") VALUES ('cafe8f46-7d7b-407e-be99-02239ab05d8c','Исайкин Георгий','Студент группы 1381','2023-11-08 23:35:31.881 +00:00','2023-11-08 23:35:31.881 +00:00'),('d4f469d0-b28d-4baa-820f-c0181bc4e47c','Тарасов Константин','Студент группы 1381','2023-11-08 23:35:31.881 +00:00','2023-11-08 23:35:31.881 +00:00'),('c41ad5cc-b1c6-418c-9458-0f17d2be413a','Васильева Ольга','Студент группы 1381','2023-11-08 23:35:31.881 +00:00','2023-11-08 23:35:31.881 +00:00'),('6abf764d-d3da-4079-b83b-74bb79b1a750','Возмитель Влас','Студент группы 1381','2023-11-08 23:35:31.881 +00:00','2023-11-08 23:35:31.881 +00:00');


^ RETURNING не дописан ^

### Определение связей
Как правило, связи в ORM нужно задавать на двух уровнях:
* Созданием внешних ключей
* Установкой связей между моделями ORM

#### Связь 1:n (A || --- o< B)

In [18]:
import { BelongsTo, ForeignKey, HasMany} from 'sequelize-typescript'

@Table
class Department extends Model {
    @BelongsTo(() => Faculty, { onDelete: 'CASCADE' })
    declare faculty: Faculty;
    
    @AllowNull(false)
    @Column(DataType.STRING(256))
    declare title: string;

    @AllowNull(false)
    @ForeignKey(() => Faculty)
    @Column(DataType.INTEGER)
    declare facultyId: number;
}

@Table
class Faculty extends Model {
    @HasMany(() => Department)
    declare departments: Department[];
    
    @AllowNull(false)
    @Column(DataType.STRING(256))
    declare title: string;
}

* `@ForeignKey` вешается на атрибут, являющийся внешним ключом
* `@BelongsTo` - кладется в сущность с внешним ключом
* `@HasMany` - кладется в сущность, на которую направлен внешний ключ
  * Есть вариант `@HasOne`

Смысл - просто создание внешних ключей не дает ORM информации о связях между моделями. Логические связи между моделями нужно установить отдельно.

Если не положить в класс `@BelongsTo` / `@HasMany`, внешний ключ будет создан, но мы не сможем нормально использовать эту связь через ORM (см. далее)

In [19]:
sequelize.addModels([Department, Faculty])
await sequelize.sync()
null

Executing (default): SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'Users'
Executing (default): SELECT i.relname AS name, ix.indisprimary AS primary, ix.indisunique AS unique, ix.indkey AS indkey, array_agg(a.attnum) as column_indexes, array_agg(a.attname) AS column_names, pg_get_indexdef(ix.indexrelid) AS definition FROM pg_class t, pg_class i, pg_index ix, pg_attribute a WHERE t.oid = ix.indrelid AND i.oid = ix.indexrelid AND a.attrelid = t.oid AND t.relkind = 'r' and t.relname = 'Users' GROUP BY i.relname, ix.indexrelid, ix.indisprimary, ix.indisunique, ix.indkey ORDER BY i.relname;
Executing (default): SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'Faculties'
Executing (default): CREATE TABLE IF NOT EXISTS "Faculties" ("id"   SERIAL , "title" VARCHAR(256) NOT NULL, "createdAt" TIMESTAMP WITH TIME ZONE NOT NULL, "updatedAt" TIMESTAMP WITH TIME ZONE NOT NULL, PRIMARY KEY ("id"));
Execu

#### Связь n:m
Foo >o --- o< Bar

In [20]:
import { BelongsToMany } from 'sequelize-typescript'

@Table
class Foo extends Model {
    @BelongsToMany(() => Bar, () => FooBar)
    declare bars: Bar[];
    
    @Column(DataType.STRING)
    declare foo: string
}

@Table
class Bar extends Model {
    @BelongsToMany(() => Foo, () => FooBar)
    declare foos: Foo[]
    
    @Column(DataType.STRING)
    declare bar: string;
}

@Table
class FooBar extends Model {
    @BelongsTo(() => Foo)
    declare foo: Foo;

    @BelongsTo(() => Bar)
    declare bar: Bar
    
    @ForeignKey(() => Foo)
    @Column(DataType.INTEGER)
    declare fooId: number;

    @ForeignKey(() => Bar)
    @Column(DataType.INTEGER)
    declare barId: number;
}

In [21]:
sequelize.addModels([Foo, Bar, FooBar])
await sequelize.sync()
null

Executing (default): SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'Users'
Executing (default): SELECT i.relname AS name, ix.indisprimary AS primary, ix.indisunique AS unique, ix.indkey AS indkey, array_agg(a.attnum) as column_indexes, array_agg(a.attname) AS column_names, pg_get_indexdef(ix.indexrelid) AS definition FROM pg_class t, pg_class i, pg_index ix, pg_attribute a WHERE t.oid = ix.indrelid AND i.oid = ix.indexrelid AND a.attrelid = t.oid AND t.relkind = 'r' and t.relname = 'Users' GROUP BY i.relname, ix.indexrelid, ix.indisprimary, ix.indisunique, ix.indkey ORDER BY i.relname;
Executing (default): SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'Faculties'
Executing (default): SELECT i.relname AS name, ix.indisprimary AS primary, ix.indisunique AS unique, ix.indkey AS indkey, array_agg(a.attnum) as column_indexes, array_agg(a.attname) AS column_names, pg_get_indexdef(ix.indexrelid

#### Несколько связей
Если в таблице имеется несколько связей на одну таблицу, при создании надо указать, какая связь относится к какому внешнему ключу:

In [22]:
@Table
class Baz extends Model {
    @BelongsTo(() => Brr, { foreignKey: 'brr1Id', onDelete: 'CASCADE' } )
    brr1: Brr

    @BelongsTo(() => Brr, { foreignKey: 'brr2Id', onDelete: 'CASCADE' })
    brr2: Brr

    @ForeignKey(() => Brr)
    @Column(DataType.INTEGER)
    brr1Id: number
    
    @ForeignKey(() => Brr)
    @Column(DataType.INTEGER)
    brr2Id: number
}

@Table
class Brr extends Model {}

In [23]:
sequelize.addModels([Baz, Brr])
await sequelize.sync()
null

Executing (default): SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'Users'
Executing (default): SELECT i.relname AS name, ix.indisprimary AS primary, ix.indisunique AS unique, ix.indkey AS indkey, array_agg(a.attnum) as column_indexes, array_agg(a.attname) AS column_names, pg_get_indexdef(ix.indexrelid) AS definition FROM pg_class t, pg_class i, pg_index ix, pg_attribute a WHERE t.oid = ix.indrelid AND i.oid = ix.indexrelid AND a.attrelid = t.oid AND t.relkind = 'r' and t.relname = 'Users' GROUP BY i.relname, ix.indexrelid, ix.indisprimary, ix.indisunique, ix.indkey ORDER BY i.relname;
Executing (default): SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'Faculties'
Executing (default): SELECT i.relname AS name, ix.indisprimary AS primary, ix.indisunique AS unique, ix.indkey AS indkey, array_agg(a.attnum) as column_indexes, array_agg(a.attname) AS column_names, pg_get_indexdef(ix.indexrelid

## Использование моделей

### Получение данных из БД

Оператор `COUNT`:

In [24]:
await User.count()

Executing (default): SELECT count(*) AS "count" FROM "Users" AS "User";
11


Выбрать все записи:

In [25]:
let res = await User.findAll()
res.length

Executing (default): SELECT "uuid", "fio", "bio", "createdAt", "updatedAt" FROM "Users" AS "User";
11


Фильтрация:

In [26]:
import { Op } from 'sequelize'
res = await User.findAll(
    {
        where: {
            fio: 'Корытов Павел Валерьевич'
        }
    }
)
console.log(res[0].fio)

res = await User.findAll(
    {
        attributes: ['fio', 'bio'],
        where: {
            fio: {[Op.iLike]: 'Корытов%'}
        }
    }
)
console.log(res.length, res[0].createdAt)

res = await User.findAll(
    {
        where: {
            [Op.or]: [
                {
                    fio: {[Op.iLike]: 'Корытов%'}
                },
                {
                    bio: {[Op.iLike]: '%Студент%'}
                }
            ]
        },
        order: [['fio', 'ASC']]
    },
)
console.log(res.length)

Executing (default): SELECT "uuid", "fio", "bio", "createdAt", "updatedAt" FROM "Users" AS "User" WHERE "User"."fio" = 'Корытов Павел Валерьевич';
Корытов Павел Валерьевич
Executing (default): SELECT "fio", "bio" FROM "Users" AS "User" WHERE "User"."fio" ILIKE 'Корытов%';
1 undefined
Executing (default): SELECT "uuid", "fio", "bio", "createdAt", "updatedAt" FROM "Users" AS "User" WHERE ("User"."fio" ILIKE 'Корытов%' OR "User"."bio" ILIKE '%Студент%') ORDER BY "User"."fio" ASC;
10


Более подробно о доступных операторах: https://sequelize.org/docs/v6/core-concepts/model-querying-basics/

Пагинация:

In [27]:
let res1 = await User.findAndCountAll({ limit: 10, offset: 0 })
console.log(res1.count, res1.rows.length)
res1 = await User.findAndCountAll({ limit: 10, offset: 10 })
console.log(res1.count, res1.rows.length)

Executing (default): SELECT count(*) AS "count" FROM "Users" AS "User";
Executing (default): SELECT "uuid", "fio", "bio", "createdAt", "updatedAt" FROM "Users" AS "User" LIMIT 10 OFFSET 0;
11 10
Executing (default): SELECT count(*) AS "count" FROM "Users" AS "User";
Executing (default): SELECT "uuid", "fio", "bio", "createdAt", "updatedAt" FROM "Users" AS "User" LIMIT 10 OFFSET 10;
11 1


### Связанные сущности

#### Создание

In [28]:
await Faculty.create({
    title: 'ФКТИ',
    departments: [
        {
            title: 'МО ЭВМ'
        },
        {
            title: 'ВТ'
        },
        {
            title: 'ИС'
        },
        {
            title: 'АПУ'
        }
    ],
}, { include: [Department] });
await Faculty.create({
    title: 'ФЭА',
    departments: [
        {
            title: 'КСУ'
        },
        {
            title: 'РАПС'
        },
        {
            title: 'САУ'
        }
    ],
}, { include: [Department] });
null

Executing (default): INSERT INTO "Faculties" ("id","title","createdAt","updatedAt") VALUES (DEFAULT,$1,$2,$3) RETURNING "id","title","createdAt","updatedAt";
Executing (default): INSERT INTO "Departments" ("id","title","facultyId","createdAt","updatedAt") VALUES (DEFAULT,$1,$2,$3,$4) RETURNING "id","title","facultyId","createdAt","updatedAt";
Executing (default): INSERT INTO "Departments" ("id","title","facultyId","createdAt","updatedAt") VALUES (DEFAULT,$1,$2,$3,$4) RETURNING "id","title","facultyId","createdAt","updatedAt";
Executing (default): INSERT INTO "Departments" ("id","title","facultyId","createdAt","updatedAt") VALUES (DEFAULT,$1,$2,$3,$4) RETURNING "id","title","facultyId","createdAt","updatedAt";
Executing (default): INSERT INTO "Departments" ("id","title","facultyId","createdAt","updatedAt") VALUES (DEFAULT,$1,$2,$3,$4) RETURNING "id","title","facultyId","createdAt","updatedAt";
Executing (default): INSERT INTO "Faculties" ("id","title","createdAt","updatedAt") VALUES (DE

Благодаря `HasMany` и `BelongsTo` можно одновременно создавать связанные сущности.

#### Выбор
В sequelize, если просто вытащить сущность, связи к ней не вытаскиваются.

In [29]:
let res2 = await Faculty.findOne({ where: { title: 'ФКТИ' }})
console.log(res2.departments)

Executing (default): SELECT "id", "title", "createdAt", "updatedAt" FROM "Faculties" AS "Faculty" WHERE "Faculty"."title" = 'ФКТИ' LIMIT 1;
undefined


В разных ORM это по-разному - в SQLAlchemy по умолчанию на моменте обращения к `res2.departments` отправился бы SQL-запрос.

Чтобы сделать `JOIN`, нужно использовать опцию `include`:

In [30]:
let res3 = await Faculty.findAll({ 
    where: { title: 'ФКТИ' },
    include: [Department]
})
console.log(res3[0].departments.length)

Executing (default): SELECT "Faculty"."id", "Faculty"."title", "Faculty"."createdAt", "Faculty"."updatedAt", "departments"."id" AS "departments.id", "departments"."title" AS "departments.title", "departments"."facultyId" AS "departments.facultyId", "departments"."createdAt" AS "departments.createdAt", "departments"."updatedAt" AS "departments.updatedAt" FROM "Faculties" AS "Faculty" LEFT OUTER JOIN "Departments" AS "departments" ON "Faculty"."id" = "departments"."facultyId" WHERE "Faculty"."title" = 'ФКТИ';
4


По умолчанию используется `LEFT JOIN`. Чтобы сделать `INNER JOIN`, надо передать `required: true`:

In [31]:
res3 = await Faculty.findAll({ 
    where: { title: 'ФКТИ' },
    include: [
        {
            model: Department,
            required: true,
            attributes: ['id', 'title'],
        }
    ]
})
console.log(res3[0].departments.length)

Executing (default): SELECT "Faculty"."id", "Faculty"."title", "Faculty"."createdAt", "Faculty"."updatedAt", "departments"."id" AS "departments.id", "departments"."title" AS "departments.title" FROM "Faculties" AS "Faculty" INNER JOIN "Departments" AS "departments" ON "Faculty"."id" = "departments"."facultyId" WHERE "Faculty"."title" = 'ФКТИ';
4


И также можно фильтровать по связанным сущностям.

In [32]:
res3 = await Faculty.findAll({ 
    where: { title: 'ФКТИ' },
    include: [
        {
            model: Department,
            required: true,
            where: {
                title: {[Op.in]: ['МО ЭВМ', 'ВТ']}
            }
        }
    ]
})
console.log(res3[0].departments.length)

Executing (default): SELECT "Faculty"."id", "Faculty"."title", "Faculty"."createdAt", "Faculty"."updatedAt", "departments"."id" AS "departments.id", "departments"."title" AS "departments.title", "departments"."facultyId" AS "departments.facultyId", "departments"."createdAt" AS "departments.createdAt", "departments"."updatedAt" AS "departments.updatedAt" FROM "Faculties" AS "Faculty" INNER JOIN "Departments" AS "departments" ON "Faculty"."id" = "departments"."facultyId" AND "departments"."title" IN ('МО ЭВМ', 'ВТ') WHERE "Faculty"."title" = 'ФКТИ';
2


В случае, когда сущности связаны несколькими связами, нужно указывать, какая связь используется:

In [33]:
await Baz.findAll({
    include: [
        {
            model: Brr,
            as: 'brr1'
        },
    ]
})

Executing (default): SELECT "Baz"."id", "Baz"."brr1Id", "Baz"."brr2Id", "Baz"."createdAt", "Baz"."updatedAt", "brr1"."id" AS "brr1.id", "brr1"."createdAt" AS "brr1.createdAt", "brr1"."updatedAt" AS "brr1.updatedAt" FROM "Bazs" AS "Baz" LEFT OUTER JOIN "Brrs" AS "brr1" ON "Baz"."brr1Id" = "brr1"."id";
[]


См. https://sequelize.org/docs/v6/advanced-association-concepts/eager-loading/

#### Удаление сущностей

Чтобы что-то удалить, можно вызвать метод `destroy` на инстансе модели:

In [34]:
let dep = await Department.findOne({ where: { title: 'ВТ' }});
await dep.destroy()
null

Executing (default): SELECT "id", "title", "facultyId", "createdAt", "updatedAt" FROM "Departments" AS "Department" WHERE "Department"."title" = 'ВТ' LIMIT 1;
Executing (default): DELETE FROM "Departments" WHERE "id" = 2


Или можно вызвать тот же метод на самой модели:

In [35]:
let res4 = await Department.destroy({ where: { title: 'ИС' }})
console.log(res4)

Executing (default): DELETE FROM "Departments" WHERE "title" = 'ИС'
1


Вывод - число удаленных записей.

### Мягкое удаление
"Мягкое удаление" - вместо удаления сущности ставить ей атрибут вроде "удалён". Sequelize поддерживает это из коробки с помощью `DeletedAt`

In [36]:
import { DeletedAt } from 'sequelize-typescript'

@Table
class Far extends Model {
    @Column(DataType.STRING)
    declare whatever: string

    @DeletedAt
    @Column(DataType.DATE)
    declare deletedAt: Date
}
sequelize.addModels([Far])
await sequelize.sync()
null

Executing (default): SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'Users'
Executing (default): SELECT i.relname AS name, ix.indisprimary AS primary, ix.indisunique AS unique, ix.indkey AS indkey, array_agg(a.attnum) as column_indexes, array_agg(a.attname) AS column_names, pg_get_indexdef(ix.indexrelid) AS definition FROM pg_class t, pg_class i, pg_index ix, pg_attribute a WHERE t.oid = ix.indrelid AND i.oid = ix.indexrelid AND a.attrelid = t.oid AND t.relkind = 'r' and t.relname = 'Users' GROUP BY i.relname, ix.indexrelid, ix.indisprimary, ix.indisunique, ix.indkey ORDER BY i.relname;
Executing (default): SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'Faculties'
Executing (default): SELECT i.relname AS name, ix.indisprimary AS primary, ix.indisunique AS unique, ix.indkey AS indkey, array_agg(a.attnum) as column_indexes, array_agg(a.attname) AS column_names, pg_get_indexdef(ix.indexrelid

In [37]:
await Far.bulkCreate([{ whatever: 'ever' }, { whatever: 'forever' }])
null

Executing (default): INSERT INTO "Fars" ("id","whatever","createdAt","updatedAt") VALUES (DEFAULT,'ever','2023-11-08 23:35:34.995 +00:00','2023-11-08 23:35:34.995 +00:00'),(DEFAULT,'forever','2023-11-08 23:35:34.995 +00:00','2023-11-08 23:35:34.995 +00:00') RETURNING "id","whatever","deletedAt","createdAt","updatedAt";


SELECT к этой табличке будет проверять `deletedAt`:

In [38]:
let far = await Far.findOne()
far.toJSON()

Executing (default): SELECT "id", "whatever", "deletedAt", "createdAt", "updatedAt" FROM "Fars" AS "Far" WHERE ("Far"."deletedAt" IS NULL) LIMIT 1;
{
  id: 1,
  whatever: 'ever',
  deletedAt: null,
  createdAt: 2023-11-08T23:35:34.995Z,
  updatedAt: 2023-11-08T23:35:34.995Z
}


Удаление будет установкой этого атрибута:

In [39]:
await far.destroy();
null

Executing (default): UPDATE "Fars" SET "deletedAt"=$1,"updatedAt"=$2 WHERE "id" = $3


Чтобы удалить по-настоящему, можно использовать force: true:

In [40]:
await far.destroy({ force: true })

Executing (default): DELETE FROM "Fars" WHERE "id" = 1
[]


И чтобы не проверять данную сущность на `deletedAt` в запросах, можно использовать `paranoid: false`

In [41]:
await Far.findAll({ paranoid: false })
await Far.findAll({ paranoid: true })
null

Executing (default): SELECT "id", "whatever", "deletedAt", "createdAt", "updatedAt" FROM "Fars" AS "Far";
Executing (default): SELECT "id", "whatever", "deletedAt", "createdAt", "updatedAt" FROM "Fars" AS "Far" WHERE ("Far"."deletedAt" IS NULL);


### Транзакции
Транзакция - набор операций, который либо выполниться полностью, либо никак.

Более подробно - см. лекция 14

In [42]:
await sequelize.transaction(async (t) => {
    const faculty = await Faculty.create({ title: 'ИФИО' }, { transaction: t });
    throw new Error("Oh no");
    await Department.create({ title: 'ФВиС', facultyId: faculty.id }, { transaction: t });
});

Executing (2ae1269f-c795-421d-b5e9-e1a6f6686665): START TRANSACTION;
Executing (2ae1269f-c795-421d-b5e9-e1a6f6686665): INSERT INTO "Faculties" ("id","title","createdAt","updatedAt") VALUES (DEFAULT,$1,$2,$3) RETURNING "id","title","createdAt","updatedAt";
Executing (2ae1269f-c795-421d-b5e9-e1a6f6686665): ROLLBACK;


Error: Oh no
    at evalmachine.<anonymous>:5:11
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async /home/pavel/20-29 Education/23 Teaching/23.01 SQL/23.01.Y23 SQL 2023/23.01.Y23.R Repos/23.01.Y23.R.01 sql-2023/orm-demo/node_modules/sequelize/lib/sequelize.js:507:18
    at async evalmachine.<anonymous>:3:22
    at async Object.execute (/home/pavel/.npm-packages/lib/node_modules/tslab/dist/executor.js:173:17)
    at async JupyterHandlerImpl.handleExecuteImpl (/home/pavel/.npm-packages/lib/node_modules/tslab/dist/jupyter.js:223:18)


In [43]:
await Faculty.findOne({ where: { title: 'ИФИО' }})

Executing (default): SELECT "id", "title", "createdAt", "updatedAt" FROM "Faculties" AS "Faculty" WHERE "Faculty"."title" = 'ИФИО' LIMIT 1;


In [44]:
await sequelize.transaction(async (t) => {
    const faculty = await Faculty.create({ title: 'ИФИО' }, { transaction: t });
    await Department.create({ title: 'ФВиС', facultyId: faculty.id }, { transaction: t });
});

Executing (a98968d7-4787-4dec-a202-6fb150b036ba): START TRANSACTION;
Executing (a98968d7-4787-4dec-a202-6fb150b036ba): INSERT INTO "Faculties" ("id","title","createdAt","updatedAt") VALUES (DEFAULT,$1,$2,$3) RETURNING "id","title","createdAt","updatedAt";
Executing (a98968d7-4787-4dec-a202-6fb150b036ba): INSERT INTO "Departments" ("id","title","facultyId","createdAt","updatedAt") VALUES (DEFAULT,$1,$2,$3,$4) RETURNING "id","title","facultyId","createdAt","updatedAt";
Executing (a98968d7-4787-4dec-a202-6fb150b036ba): COMMIT;


## Raw SQL

Иногда может требоваться написать запрос на чистом SQL. Это может быть необходимо, если:
* Недостаточно мощности ORM
* Имеются проблемы с производительностью

### Полностью Raw SQL
sequelize поддерживает исполнение произвольных запросов с помощью метода `query`.

In [45]:
import { QueryTypes } from 'sequelize'
await sequelize.query('SELECT 2 + 3 res', { type: QueryTypes.SELECT });

Executing (default): SELECT 2 + 3 res
[ { res: 5 } ]


Можно передать параметры в запрос:

In [46]:
let needle = 'Корытов'
await sequelize.query(
    'SELECT * FROM "Users" where fio like :needle', 
    { type: QueryTypes.SELECT, replacements: { needle: needle + '%' } }
);

Executing (default): SELECT * FROM "Users" where fio like 'Корытов%'
[
  {
    uuid: 'c17bc685-8775-4a88-9035-3e1d8f3ee4a9',
    fio: 'Корытов Павел Валерьевич',
    bio: 'Программист ОИС',
    createdAt: 2023-11-08T23:35:31.193Z,
    updatedAt: 2023-11-08T23:35:31.268Z
  }
]


Как видно, возращаются простые объекты js, не инстансы моделей.

Никогда не делайте так, если не уверены в параметрах (например, если они приходят от пользователей):

In [47]:
await sequelize.query(
    `SELECT * FROM "Users" where fio like '${needle}%'`, 
    { type: QueryTypes.SELECT }
)

Executing (default): SELECT * FROM "Users" where fio like 'Корытов%'
[
  {
    uuid: 'c17bc685-8775-4a88-9035-3e1d8f3ee4a9',
    fio: 'Корытов Павел Валерьевич',
    bio: 'Программист ОИС',
    createdAt: 2023-11-08T23:35:31.193Z,
    updatedAt: 2023-11-08T23:35:31.268Z
  }
]


Потому что:

In [48]:
needle = `1' OR 1 = 1 --`
await sequelize.query(
    `SELECT * FROM "Users" where fio like '${needle}%'`, 
    { type: QueryTypes.SELECT }
)

Executing (default): SELECT * FROM "Users" where fio like '1' OR 1 = 1 --%'
[
  {
    uuid: 'c17bc685-8775-4a88-9035-3e1d8f3ee4a9',
    fio: 'Корытов Павел Валерьевич',
    bio: 'Программист ОИС',
    createdAt: 2023-11-08T23:35:31.193Z,
    updatedAt: 2023-11-08T23:35:31.268Z
  },
  {
    uuid: '0f9084f1-01b6-483e-91e5-a22e8cc63f4d',
    fio: 'Азаревич Артём Дмитриевич',
    bio: 'Аспирант каф. МОЭВМ',
    createdAt: 2023-11-08T23:35:31.494Z,
    updatedAt: 2023-11-08T23:35:31.521Z
  },
  {
    uuid: '626e53d9-84d9-4964-b1bd-fb95ff807ede',
    fio: 'Депрейс Александр',
    bio: 'Студент группы 1303',
    createdAt: 2023-11-08T23:35:31.718Z,
    updatedAt: 2023-11-08T23:35:31.718Z
  },
  {
    uuid: '1717ca0e-4d7e-4921-8e99-2ec72dcc87c5',
    fio: 'Коренев Данил',
    bio: 'Студент группы 1303',
    createdAt: 2023-11-08T23:35:31.718Z,
    updatedAt: 2023-11-08T23:35:31.718Z
  },
  {
    uuid: 'f82eeb99-fc89-4b42-b302-c3e0f2116c22',
    fio: 'Кузнецов Николай',
    bio: 'Студент группы

И это далеко не самый страшный вариант, что можно сделать с таким запросом.

### Использование элементов Raw SQL в запросах Sequelize
sequelize также позволяет встроить "кусок" чистого SQL в свои запросы.

Это в документации практически не описано.

In [49]:
import { literal } from 'sequelize'

In [50]:
let res5 = await User.findAll({
    attributes: {
        // Добавление дополнительного атрибута.
        include: [[literal('length("fio")'), 'fioLen']]
        // Тут можно позвать и SELECT (но обычно не нужно)
    },
    where: {
        [Op.and]: [
            // Кастомное условие для WHERE
            literal(`split_part(fio, ' ', 3) = 'Валерьевич'`), 
        ]
    },
    // Кастомный ORDER
    order: literal(`RANDOM()`)
})
res5[0].toJSON()

Executing (default): SELECT "uuid", "fio", "bio", "createdAt", "updatedAt", length("fio") AS "fioLen" FROM "Users" AS "User" WHERE (split_part(fio, ' ', 3) = 'Валерьевич') ORDER BY RANDOM();
{
  uuid: 'c17bc685-8775-4a88-9035-3e1d8f3ee4a9',
  fio: 'Корытов Павел Валерьевич',
  bio: 'Программист ОИС',
  createdAt: 2023-11-08T23:35:31.193Z,
  updatedAt: 2023-11-08T23:35:31.268Z,
  fioLen: 24
}


Важно, что обратиться к атрибутам, которых не существует в модели, так не получится:

In [51]:
res5[0].fioLen

1:9 - Property 'fioLen' does not exist on type 'User'.


Можно использовать `getDataValue()`:

In [52]:
res5[0].getDataValue('fioLen')

24


Главное - не делать так:

In [53]:
needle = `1' OR 1 = 1 OR '1' = '1`
res5 = await User.findAll({
    where: {
        [Op.and]: [
            literal(`split_part(fio, ' ', 3) = '${needle}'`), 
        ]
    },
})
res5.length

Executing (default): SELECT "uuid", "fio", "bio", "createdAt", "updatedAt" FROM "Users" AS "User" WHERE (split_part(fio, ' ', 3) = '1' OR 1 = 1 OR '1' = '1');
11


Можно сделать так:

In [54]:
res5 = await User.findAll({
    where: {
        [Op.and]: [
            literal(`split_part(fio, ' ', 3) = ?`), 
        ]
    },
    replacements: [needle]
})

Executing (default): SELECT "uuid", "fio", "bio", "createdAt", "updatedAt" FROM "Users" AS "User" WHERE (split_part(fio, ' ', 3) = '1'' OR 1 = 1 OR ''1'' = ''1');
[]


### Использование элементов Sequelize в Raw SQL
Можно поступить и наоборот - использовать логику sequelize для генерации части запроса.

Это в документации не описано совсем :-)

In [55]:
let qgen = sequelize.getQueryInterface().queryGenerator as any

In [56]:
console.log(
    qgen.whereQuery({ fio: { [Op.iLike]: 'Корытов%' } })
)
console.log(
    qgen.whereItemsQuery({ fio: { [Op.iLike]: 'Корытов%' } })
)

WHERE "fio" ILIKE 'Корытов%'
"fio" ILIKE 'Корытов%'


In [57]:
console.log(
    qgen.getQueryOrders({ order: [['fio', 'ASC'], ['uuid', 'DESC']] }).mainQueryOrder.join(', ')
)

"fio" ASC, "uuid" DESC


In [58]:
console.log(
    qgen.addLimitAndOffset({ limit: 10, offset: 0 })
)

 LIMIT 10 OFFSET 0


Поэтому можно так делать, если другого выхода нет:

In [59]:
let whereQuery = qgen.whereQuery({ fio: { [Op.iLike]: 'Корытов%' } })
let orderQuery = qgen.getQueryOrders({ order: [['fio', 'ASC'], ['uuid', 'DESC']] }).mainQueryOrder.join(', ')
let paginationQuery = qgen.addLimitAndOffset({ limit: 10, offset: 0 })
let query = `
SELECT * FROM "Users"
${whereQuery}
ORDER BY ${orderQuery}
${paginationQuery}
`
await sequelize.query(query, { type: QueryTypes.SELECT })

Executing (default): SELECT * FROM "Users"
WHERE "fio" ILIKE 'Корытов%'
ORDER BY "fio" ASC, "uuid" DESC
 LIMIT 10 OFFSET 0
[
  {
    uuid: 'c17bc685-8775-4a88-9035-3e1d8f3ee4a9',
    fio: 'Корытов Павел Валерьевич',
    bio: 'Программист ОИС',
    createdAt: 2023-11-08T23:35:31.193Z,
    updatedAt: 2023-11-08T23:35:31.268Z
  }
]
